In [1]:
import config
import os, numpy as np, pandas, sklearn
import mido
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## NN libs
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
import keras
from keras import regularizers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Activation, Conv1D, Conv2D, Dropout, Flatten
from keras.layers import Conv2DTranspose, Reshape, MaxPooling2D, UpSampling2D, UpSampling1D, MaxPooling1D
from keras.layers import LocallyConnected1D, LocallyConnected2D
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from keras.callbacks import TensorBoard

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# local libs
import config, models, functions
from data import data, midi
from utils import io, models_io

 Context :: namedtuple(
[ max_t = float
, dt = float
, n_instances = int
, note_length = int
, bpm = float
, tempo = float
, ticks_per_beat = int
]



In [4]:
n: int = 8
context, x_train, labels = data.init(n)
print(x_train.shape)

Setting up params

 >> Context(max_t=10.0, dt=0.01, n_instances=1000, note_length=0.03, bpm=120.0, tempo=500000, ticks_per_beat=480)
Importing midi-data

[INFO] :
 |  reading file: ../datasets/examples/01 16th Snare.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Cym.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Hat.mid
[INFO] :
 |  reading file: ../datasets/examples/02 8th Cym.mid
[INFO] :
 |  reading file: ../datasets/examples/114_LetsDance_DavidBowie1.mid
[INFO] :
 |  reading file: ../datasets/examples/127_SweetChildOMine_GunsAndRoses1.mid
[INFO] :
 |  reading file: ../datasets/examples/92_GiveItAway_TheRedHotChiliPeppers.mid
[INFO] :
 |  reading file: ../datasets/examples/94_Creep_Radiohead2.mid

Encoding midi-data
 [<midi file '../datasets/examples/01 16th Snare.mid' type 0, 1 tracks, 182 messages>, <midi file '../datasets/examples/01 8th Cym.mid' type 0, 1 tracks, 68 messages>, <midi file '../datasets/examples/01 8th Hat.mid' type 0, 1 tracks, 60 messages>

In [14]:
fn = (config.dataset_dir + '4-floor-120bpm.mid')
mid = io.import_midifile(fn)

[INFO] :
 |  reading file: ../datasets/4-floor-120bpm.mid


## Setup a generator

In [50]:
n = 100
x_train = np.array([[0]] * n)
y_train = np.stack([midi.encode(context, mid) for _ in range(n)])

In [51]:
x_train.shape

(100, 1)

In [53]:
n_samples = x_train[0]
input_shape = x_train.shape[1:] # shape of a single sample
output_shape = y_train.shape[1:]
# output_length = y_train.shape[1]
# output_length = (y_train[0]).shape[0] # = length of an individual label
hidden_layer_length = input_shape[0]
x_train.shape

(100, 1)

In [54]:
def decoder(input_length, output_shape, dropout=0.10):
    input_layer = Input(shape=(input_length,))
    x = input_layer
    shape = (10,100) # 1 additional dimension
    x = Dense(np.prod(shape), activation='relu')(x) # 4*4*8 = 128
    x = Reshape(shape)(x)
    x = UpSampling1D(10)(x)
    x = Conv1D(100, 2, strides=2, activation='relu')(x) # 50,100
    x = UpSampling1D(output_shape[0] / 50)(x)
    x = Dense(output_shape[1], activation='relu')(x)
#     x = Dense(output_shape[1], activation='relu')(x)
#     x = Dense(output_shape[1], activation='relu')(x)
    #     x = LocallyConnected1D(output_shape[1], kernel_size=1, activation='relu')(x)
    # x = Dense(output_length, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=x)
    #     model.add(Dropout(dropout))
    return model, model.summary

In [55]:
model, summary = decoder(hidden_layer_length, output_shape)
summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 1000)              2000      
_________________________________________________________________
reshape_5 (Reshape)          (None, 10, 100)           0         
_________________________________________________________________
up_sampling1d_9 (UpSampling1 (None, 100, 100)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 50, 100)           20100     
_________________________________________________________________
up_sampling1d_10 (UpSampling (None, 1000, 100)         0         
_________________________________________________________________
dense_14 (Dense)             (None, 1000, 127)         12827     
Total para

In [56]:
metrics = ['accuracy','mse','categorical_crossentropy']
loss = 'binary_crossentropy'# binary_crossentropy categorical_crossentropy
optimizer = 'adadelta'
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [57]:
batch_size = 16
# n epochs = n iterations over all the training data
epochs = 50

In [58]:
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[TensorBoard(log_dir=config.tmp_log_dir)])
# model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
#           validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

Epoch 1/50
100/100 [==============================] - 2s 20ms/step - loss: 0.0010 - acc: 0.9999 - mean_squared_error: 6.2992e-05 - categorical_crossentropy: nan
Epoch 2/50
100/100 [==============================] - 1s 13ms/step - loss: 0.0010 - acc: 0.9999 - mean_squared_error: 6.2992e-05 - categorical_crossentropy: nan
Epoch 3/50
100/100 [==============================] - 1s 14ms/step - loss: 0.0010 - acc: 0.9999 - mean_squared_error: 6.2992e-05 - categorical_crossentropy: nan
Epoch 4/50
100/100 [==============================] - 1s 14ms/step - loss: 0.0010 - acc: 0.9999 - mean_squared_error: 6.2992e-05 - categorical_crossentropy: nan
Epoch 5/50
100/100 [==============================] - 1s 14ms/step - loss: 0.0010 - acc: 0.9999 - mean_squared_error: 6.2992e-05 - categorical_crossentropy: nan
Epoch 6/50
100/100 [==============================] - 1s 15ms/step - loss: 0.0010 - acc: 0.9999 - mean_squared_error: 6.2992e-05 - categorical_crossentropy: nan
Epoch 7/50
100/100 [==============